### IMPORT CÁC THƯ VIỆN

In [1]:
from datetime import datetime, timedelta,date
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
from sklearn.cluster import KMeans
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
#Sử dụng model xgboost
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [2]:
pyoff.init_notebook_mode()
tx_data =pd.read_csv('data.csv')
tx_data['InvoiceDate'] = pd.to_datetime(tx_data['InvoiceDate'])
tx_uk = tx_data.query("Country == 'United Kingdom'").reset_index(drop=True)
tx_uk

InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
495473    581585     22466       FAIRY TALE COTTAGE NIGHT LIGHT        12   
495474    581586     22061  LARGE CAKE STAND  HANGING STRAWBERY         8   
495475    581586     23275     SET OF 3 HANGING OWLS OLLIE BEAK        24   
495476    581586     21217        RED RETROSPOT ROUND CAKE TINS        24   
495477    581586     20685                DOORMAT RED RETROSPOT        10   

               InvoiceDate  UnitPrice  CustomerID         Country  
0      2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1      2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2      2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3      2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4      2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
...                    ...        ...         ...             ...  
495473 2011-12-09 12:31:00       1.95     15804.0  United Kingdom  
495474 2011-12-09 12:49:00       2.95     13113.0  United Kingdom  
495475 2011-12-09 12:49:00       1.25     13113.0  United Kingdom  
495476 2011-12-09 12:49:00       8.95     13113.0  United Kingdom  
495477 2011-12-09 12:49:00       7.08     13113.0  United Kingdom  

[495478 rows x 8 columns]

#### Chia dữ liệu thành 2 khung thời gian, 3 thán để tính RFM làm feature co model, 6 tháng để test

In [3]:
tx_3m =tx_uk[(tx_uk.InvoiceDate < datetime(2011,6,1)) & (tx_uk.InvoiceDate >= datetime(2011,3,1))].reset_index(drop=True)
tx_6m = tx_uk[(tx_uk.InvoiceDate >= datetime(2011,6,1)) & (tx_uk.InvoiceDate < datetime(2011,12,1))].reset_index(drop=True)

In [4]:
tx_3m

InvoiceNo StockCode                          Description  Quantity  \
0        545220     21955    DOORMAT UNION JACK GUNS AND ROSES         2   
1        545220     48194                       DOORMAT HEARTS         2   
2        545220     22556       PLASTERS IN TIN CIRCUS PARADE         12   
3        545220     22139     RETROSPOT TEA SET CERAMIC 11 PC          3   
4        545220    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         4   
...         ...       ...                                  ...       ...   
95188    555150     22895  SET OF 2 TEA TOWELS APPLE AND PEARS         3   
95189    555150     22983                  CARD BILLBOARD FONT         3   
95190    555150     23169             CLASSIC GLASS COOKIE JAR         3   
95191    555150     23298                       SPOTTY BUNTING         5   
95192    555150     23118         PARISIENNE JEWELLERY DRAWER          2   

              InvoiceDate  UnitPrice  CustomerID         Country  
0     2011-03-01 08:30:00       7.95     14620.0  United Kingdom  
1     2011-03-01 08:30:00       7.95     14620.0  United Kingdom  
2     2011-03-01 08:30:00       1.65     14620.0  United Kingdom  
3     2011-03-01 08:30:00       4.95     14620.0  United Kingdom  
4     2011-03-01 08:30:00       3.75     14620.0  United Kingdom  
...                   ...        ...         ...             ...  
95188 2011-05-31 15:53:00       3.25         NaN  United Kingdom  
95189 2011-05-31 15:53:00       0.42         NaN  United Kingdom  
95190 2011-05-31 15:53:00       4.15         NaN  United Kingdom  
95191 2011-05-31 15:53:00       4.95         NaN  United Kingdom  
95192 2011-05-31 15:53:00       7.50         NaN  United Kingdom  

[95193 rows x 8 columns]

In [5]:
tx_6m

InvoiceNo StockCode                          Description  Quantity  \
0         555156     23299         FOOD COVER WITH BEADS SET 2          6   
1         555156     22847          BREAD BIN DINER STYLE IVORY         1   
2         555157     23075            PARLOUR CERAMIC WALL HOOK        16   
3         555157    47590B          PINK HAPPY BIRTHDAY BUNTING         6   
4         555157     22423             REGENCY CAKESTAND 3 TIER         4   
...          ...       ...                                  ...       ...   
278961   C579886     22197                       POPCORN HOLDER        -1   
278962   C579886     23146       TRIPLE HOOK ANTIQUE IVORY ROSE        -1   
278963   C579887     84946         ANTIQUE SILVER T-LIGHT GLASS        -1   
278964   C579887     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        -1   
278965   C579887     23490     T-LIGHT HOLDER HANGING LOVE BIRD        -3   

               InvoiceDate  UnitPrice  CustomerID         Country  
0      2011-06-01 07:37:00       3.75     15643.0  United Kingdom  
1      2011-06-01 07:37:00      16.95     15643.0  United Kingdom  
2      2011-06-01 07:38:00       4.15     15643.0  United Kingdom  
3      2011-06-01 07:38:00       5.45     15643.0  United Kingdom  
4      2011-06-01 07:38:00      12.75     15643.0  United Kingdom  
...                    ...        ...         ...             ...  
278961 2011-11-30 17:39:00       0.85     15676.0  United Kingdom  
278962 2011-11-30 17:39:00       3.29     15676.0  United Kingdom  
278963 2011-11-30 17:42:00       1.25     16717.0  United Kingdom  
278964 2011-11-30 17:42:00       7.95     16717.0  United Kingdom  
278965 2011-11-30 17:42:00       3.75     16717.0  United Kingdom  

[278966 rows x 8 columns]

### 3 Month tính điểm RFM

In [6]:
tx_user =pd.DataFrame(tx_3m['CustomerID'].unique())
tx_user.columns =['CustomerID']
tx_user.head()

CustomerID
0     14620.0
1     14740.0
2     13880.0
3     16462.0
4     17068.0

In [7]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new =df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new =df_new.sort_values(by=target_field_name, ascending=ascending).reset_index(drop=True)
    df_new['index']=df_new.index
    df_final =pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name], axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [8]:
tx_max_purchase =tx_3m.groupby('CustomerID').InvoiceDate.max().reset_index()
tx_max_purchase.columns = ['CustomerID', 'MaxPurchaseDate']
tx_max_purchase['Recency']=(tx_max_purchase['MaxPurchaseDate'].max()-tx_max_purchase['MaxPurchaseDate']).dt.days
tx_user = pd.merge(tx_user, tx_max_purchase[['CustomerID','Recency']], on='CustomerID')

In [9]:
kmeans =KMeans(n_clusters=4)
kmeans.fit(tx_user[['Recency']])
tx_user['RecencyCluster'] =kmeans.predict(tx_user[['Recency']])
tx_user = order_cluster('RecencyCluster', 'Recency',tx_user, False)

In [10]:
tx_user.groupby('RecencyCluster')['Recency'].describe()

count       mean       std   min   25%   50%   75%   max
RecencyCluster                                                          
0               349.0  76.885387  8.373023  64.0  70.0  76.0  84.0  91.0
1               439.0  50.020501  7.377743  39.0  43.0  49.0  56.0  63.0
2               444.0  24.373874  4.959870  17.0  20.0  23.0  27.0  35.0
3               608.0   8.312500  4.404323   0.0   5.0   8.0  12.0  16.0

In [11]:
tx_frequency= tx_3m.groupby('CustomerID').InvoiceDate.count().reset_index()
tx_frequency.columns =['CustomerID','Frequency']
tx_user =pd.merge(tx_user, tx_frequency, on='CustomerID')
kmeans=KMeans(n_clusters=4)
kmeans.fit(tx_user[['Frequency']])
tx_user['FrequencyCluster']=kmeans.predict(tx_user[['Frequency']])
tx_user = order_cluster('FrequencyCluster','Frequency',tx_user,True)
tx_user.groupby('FrequencyCluster')['Frequency'].describe()

count         mean         std     min     25%     50%  \
FrequencyCluster                                                            
0                 1605.0    24.218069   17.761893     1.0    10.0    20.0   
1                  223.0   116.560538   43.816927    71.0    83.5   104.0   
2                   11.0   496.000000  125.912668   331.0   422.5   452.0   
3                    1.0  1364.000000         NaN  1364.0  1364.0  1364.0   

                     75%     max  
FrequencyCluster                  
0                   36.0    70.0  
1                  131.5   285.0  
2                  570.0   730.0  
3                 1364.0  1364.0

In [12]:
tx_3m['Revenue']=tx_3m['UnitPrice']*tx_3m['Quantity']
tx_revenue = tx_3m.groupby('CustomerID').Revenue.sum().reset_index()
tx_user =pd.merge(tx_user, tx_revenue , on ='CustomerID')
kmeans=KMeans(n_clusters=4)
kmeans.fit(tx_user[['Revenue']])
tx_user['RevenueCluster']= kmeans.predict(tx_user[['Revenue']])
tx_user= order_cluster('RevenueCluster','Revenue',tx_user, True)
tx_user.groupby('RevenueCluster')['Revenue'].describe()

count          mean          std       min         25%  \
RevenueCluster                                                            
0               1756.0    504.760245   444.567580  -1462.50    204.3075   
1                 72.0   3760.423486  1538.877714   2159.16   2685.1875   
2                 10.0  14092.686000  2855.109881  10348.13  11268.3675   
3                  2.0  30599.645000  6343.928696  26113.81  28356.7275   

                      50%         75%       max  
RevenueCluster                                   
0                 349.590    666.8325   2131.72  
1                3141.610   4222.5050   7969.99  
2               15124.345  16124.8250  17700.64  
3               30599.645  32842.5625  35085.48

In [13]:
tx_user['OverallScore']=tx_user['RecencyCluster']+tx_user['FrequencyCluster']+tx_user['RevenueCluster']
tx_user.groupby('OverallScore')['Recency','Frequency','Revenue'].mean()

<ipython-input-13-f9d8cbe33366>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



Recency   Frequency       Revenue
OverallScore                                     
0             77.014925   18.432836    295.824239
1             50.887828   24.522673    367.334177
2             26.590476   31.028571    500.973095
3             10.243902   36.652439    633.871402
4              8.109375  103.835938   1983.985086
5              6.482759  138.965517   4326.155172
6              7.357143  322.071429  12177.627143
7              3.000000  779.333333  11350.896667

In [23]:
tx_user['Segment'] = 'Low-Value'
tx_user.loc[tx_user['OverallScore']>2,'Segment'] = 'Mid-Value' 
tx_user.loc[tx_user['OverallScore']>4,'Segment'] = 'High-Value'

In [24]:
tx_user.groupby('OverallScore')['Recency'].count()

OverallScore
0    335
1    419
2    420
3    492
4    128
5     29
6     14
7      3
Name: Recency, dtype: int64

In [25]:
tx_user.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  Revenue  \
0     14620.0       12               3         30                 0   393.28   
1     15194.0        6               3         64                 0  1439.02   
2     18044.0        5               3         57                 0   808.96   
3     18075.0       12               3         35                 0   638.12   
4     15241.0        0               3         64                 0   947.55   

   RevenueCluster  OverallScore    Segment  
0               0             3  Mid-Value  
1               0             3  Mid-Value  
2               0             3  Mid-Value  
3               0             3  Mid-Value  
4               0             3  Mid-Value

### 6 Month

In [26]:
tx_6m

InvoiceNo StockCode                          Description  Quantity  \
0         555156     23299         FOOD COVER WITH BEADS SET 2          6   
1         555156     22847          BREAD BIN DINER STYLE IVORY         1   
2         555157     23075            PARLOUR CERAMIC WALL HOOK        16   
3         555157    47590B          PINK HAPPY BIRTHDAY BUNTING         6   
4         555157     22423             REGENCY CAKESTAND 3 TIER         4   
...          ...       ...                                  ...       ...   
278961   C579886     22197                       POPCORN HOLDER        -1   
278962   C579886     23146       TRIPLE HOOK ANTIQUE IVORY ROSE        -1   
278963   C579887     84946         ANTIQUE SILVER T-LIGHT GLASS        -1   
278964   C579887     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        -1   
278965   C579887     23490     T-LIGHT HOLDER HANGING LOVE BIRD        -3   

               InvoiceDate  UnitPrice  CustomerID         Country  Revenue  
0      2011-06-01 07:37:00       3.75     15643.0  United Kingdom    22.50  
1      2011-06-01 07:37:00      16.95     15643.0  United Kingdom    16.95  
2      2011-06-01 07:38:00       4.15     15643.0  United Kingdom    66.40  
3      2011-06-01 07:38:00       5.45     15643.0  United Kingdom    32.70  
4      2011-06-01 07:38:00      12.75     15643.0  United Kingdom    51.00  
...                    ...        ...         ...             ...      ...  
278961 2011-11-30 17:39:00       0.85     15676.0  United Kingdom    -0.85  
278962 2011-11-30 17:39:00       3.29     15676.0  United Kingdom    -3.29  
278963 2011-11-30 17:42:00       1.25     16717.0  United Kingdom    -1.25  
278964 2011-11-30 17:42:00       7.95     16717.0  United Kingdom    -7.95  
278965 2011-11-30 17:42:00       3.75     16717.0  United Kingdom   -11.25  

[278966 rows x 9 columns]

In [27]:
# tính tổng doanh thu từ mỗi khách hàng
tx_6m['Revenue'] = tx_6m['UnitPrice']*tx_6m['Quantity']
tx_user_6m =tx_6m.groupby('CustomerID')['Revenue'].sum().reset_index()
tx_user_6m.columns = ['CustomerID','m6_Revenue']
plot_data = [
    go.Histogram(
        x=tx_user_6m.query('m6_Revenue < 10000')['m6_Revenue']
    )
]

plot_layout= go.Layout(
    title='6m Revenue'
)
fig= go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

#### có những khách hàng mang lại giá trị âm do chương trình khuyến mãi,.... tập dữ liệu bao gồm cả chi phí cho khách hàng

In [28]:
tx_merge =pd.merge (tx_user, tx_user_6m, on = 'CustomerID', how='left')
# kết hợp data 3m và 6m để thấy sự tương quan giữa RFM 3 tháng và Revenue 6m
tx_merge =tx_merge.fillna(0)
# điền vào các ô NaN là 0
tx_graph = tx_merge.query("m6_Revenue < 30000")
plot_data =[
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Low-Value'")['m6_Revenue'],
        mode='markers',
        name='Low',
        marker= dict (size=7,
                     line=dict(width=1),
                     color='blue',
                     opacity=0.8
                     )
    ),
    go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Mid-Value'")['m6_Revenue'],
        mode='markers',
        name='Mid',
        marker= dict (size=9,
                     line=dict(width=1),
                     color='green',
                     opacity=0.5
                     )
    ),
    go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'High-Value'")['m6_Revenue'],
        mode='markers',
        name='High',
    marker=dict(size=11,
               line=dict(width=1),
                color='red',
                opacity=0.9
               )
    ),
]
plot_layout = go.Layout(
    xaxis={'title':"RFM Score"},
    yaxis={'title':"6m LTV"},
    title='LTV'
)
fig =go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

#### Sự tương quan khá tốt điểm RFM cao giá trị Revenue cao tương ứng

In [29]:
# Phân cụm các nhóm LTV cao, thấp, trung bình
tx_merge = tx_merge[tx_merge['m6_Revenue']<tx_merge['m6_Revenue'].quantile(0.99)]
kmeans=KMeans(n_clusters=3)
kmeans.fit(tx_merge[['m6_Revenue']])
tx_merge['LTVCluster'] = kmeans.predict(tx_merge[['m6_Revenue']])
tx_merge = order_cluster('LTVCluster', 'm6_Revenue',tx_merge,True)
tx_cluster = tx_merge.copy()
tx_cluster.groupby('LTVCluster')['m6_Revenue'].describe()

count         mean          std      min       25%       50%  \
LTVCluster                                                                  
0           1396.0   397.640746   421.465554  -609.40     0.000   296.410   
1            369.0  2498.470678   936.692218  1448.78  1738.210  2164.710   
2             56.0  8222.565893  2983.572030  5396.44  6151.435  6986.545   

                  75%       max  
LTVCluster                       
0            686.6025   1445.93  
1           3052.4600   5287.39  
2           9607.3225  16756.31

In [34]:
tx_cluster.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  Revenue  \
0     14620.0       12               3         30                 0   393.28   
1     18044.0        5               3         57                 0   808.96   
2     18075.0       12               3         35                 0   638.12   
3     15241.0        0               3         64                 0   947.55   
4     15660.0        4               3         34                 0   484.62   

   RevenueCluster  OverallScore    Segment  m6_Revenue  LTVCluster  
0               0             3  Mid-Value        0.00           0  
1               0             3  Mid-Value      991.54           0  
2               0             3  Mid-Value     1322.75           0  
3               0             3  Mid-Value      791.04           0  
4               0             3  Mid-Value      858.09           0

In [30]:
tx_class =pd.get_dummies(tx_cluster)
# chuyển đổi biến categorical thành biến dummy/ indicator 

In [36]:
tx_class.head()

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  Revenue  \
0     14620.0       12               3         30                 0   393.28   
1     18044.0        5               3         57                 0   808.96   
2     18075.0       12               3         35                 0   638.12   
3     15241.0        0               3         64                 0   947.55   
4     15660.0        4               3         34                 0   484.62   

   RevenueCluster  OverallScore  m6_Revenue  LTVCluster  Segment_High-Value  \
0               0             3        0.00           0                   0   
1               0             3      991.54           0                   0   
2               0             3     1322.75           0                   0   
3               0             3      791.04           0                   0   
4               0             3      858.09           0                   0   

   Segment_Low-Value  Segment_Mid-Value  
0                  0                  1  
1                  0                  1  
2                  0                  1  
3                  0                  1  
4                  0                  1

In [38]:
corr_matrix=tx_class.corr()
# tìm mối tương quan theo cặp của tất cả các cột trong dataframe
corr_matrix['LTVCluster'].sort_values(ascending=False)
# sắp xếp lại score
corr_matrix

CustomerID   Recency  RecencyCluster  Frequency  \
CustomerID            1.000000 -0.002528        0.006726  -0.041699   
Recency              -0.002528  1.000000       -0.965208  -0.255674   
RecencyCluster        0.006726 -0.965208        1.000000   0.250344   
Frequency            -0.041699 -0.255674        0.250344   1.000000   
FrequencyCluster     -0.001735 -0.217506        0.209192   0.789243   
Revenue              -0.059981 -0.310914        0.313290   0.493238   
RevenueCluster       -0.043978 -0.176933        0.174159   0.340535   
OverallScore         -0.001481 -0.914461        0.941717   0.477430   
m6_Revenue           -0.039158 -0.250921        0.250437   0.374258   
LTVCluster           -0.027692 -0.239359        0.238502   0.358736   
Segment_High-Value   -0.051896 -0.153673        0.160735   0.470076   
Segment_Low-Value     0.005218  0.731302       -0.807441  -0.334609   
Segment_Mid-Value     0.009580 -0.696642        0.771732   0.204241   

                    FrequencyCluster   Revenue  RevenueCluster  OverallScore  \
CustomerID                 -0.001735 -0.059981       -0.043978     -0.001481   
Recency                    -0.217506 -0.310914       -0.176933     -0.914461   
RecencyCluster              0.209192  0.313290        0.174159      0.941717   
Frequency                   0.789243  0.493238        0.340535      0.477430   
FrequencyCluster            1.000000  0.457022        0.310259      0.493382   
Revenue                     0.457022  1.000000        0.785147      0.511996   
RevenueCluster              0.310259  0.785147        1.000000      0.386682   
OverallScore                0.493382  0.511996        0.386682      1.000000   
m6_Revenue                  0.377632  0.738058        0.620276      0.411409   
LTVCluster                  0.364389  0.600225        0.468079      0.374104   
Segment_High-Value          0.419725  0.577521        0.681262      0.354948   
Segment_Low-Value          -0.351350 -0.364761       -0.240919     -0.824357   
Segment_Mid-Value           0.235619  0.204002        0.048860      0.733234   

                    m6_Revenue  LTVCluster  Segment_High-Value  \
CustomerID           -0.039158   -0.027692           -0.051896   
Recency              -0.250921   -0.239359           -0.153673   
RecencyCluster        0.250437    0.238502            0.160735   
Frequency             0.374258    0.358736            0.470076   
FrequencyCluster      0.377632    0.364389            0.419725   
Revenue               0.738058    0.600225            0.577521   
RevenueCluster        0.620276    0.468079            0.681262   
OverallScore          0.411409    0.374104            0.354948   
m6_Revenue            1.000000    0.846302            0.475710   
LTVCluster            0.846302    1.000000            0.353042   
Segment_High-Value    0.475710    0.353042            1.000000   
Segment_Low-Value    -0.284336   -0.265620           -0.185806   
Segment_Mid-Value     0.151711    0.167893           -0.098259   

                    Segment_Low-Value  Segment_Mid-Value  
CustomerID                   0.005218           0.009580  
Recency                      0.731302          -0.696642  
RecencyCluster              -0.807441           0.771732  
Frequency                   -0.334609           0.204241  
FrequencyCluster            -0.351350           0.235619  
Revenue                     -0.364761           0.204002  
RevenueCluster              -0.240919           0.048860  
OverallScore                -0.824357           0.733234  
m6_Revenue                  -0.284336           0.151711  
LTVCluster                  -0.265620           0.167893  
Segment_High-Value          -0.185806          -0.098259  
Segment_Low-Value            1.000000          -0.959574  
Segment_Mid-Value           -0.959574           1.000000

In [39]:
X=tx_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y=tx_class['LTVCluster']
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.05, random_state=56)
# chia tập dữ liệu train,test 0.05 

In [45]:
X_train

CustomerID  Recency  RecencyCluster  Frequency  FrequencyCluster  \
18       15220.0       11               3         58                 0   
491      15376.0       30               2         59                 0   
1447     13004.0        8               3         62                 0   
323      17345.0        4               3         22                 0   
938      13652.0       64               0         20                 0   
...          ...      ...             ...        ...               ...   
1259     15625.0       43               1         36                 0   
1442     13571.0        0               3         58                 0   
1216     14154.0       46               1         52                 0   
399      15815.0       20               2         19                 0   
484      15210.0       32               2         20                 0   

      Revenue  RevenueCluster  OverallScore  Segment_High-Value  \
18     838.15               0             3                   0   
491    654.17               0             2                   0   
1447   840.18               0             3                   0   
323    128.33               0             3                   0   
938    329.44               0             0                   0   
...       ...             ...           ...                 ...   
1259   695.33               0             1                   0   
1442   392.60               0             3                   0   
1216  1345.62               0             1                   0   
399   1394.08               0             2                   0   
484    331.18               0             2                   0   

      Segment_Low-Value  Segment_Mid-Value  
18                    0                  1  
491                   1                  0  
1447                  0                  1  
323                   0                  1  
938                   1                  0  
...                 ...                ...  
1259                  1                  0  
1442                  0                  1  
1216                  1                  0  
399                   1                  0  
484                   1                  0  

[1729 rows x 11 columns]

In [46]:
# Đặt các tham số cho model
ltv_xgb_model = xgb.XGBClassifier(max_depth=5, learnig_rate=0.1, objective='multi:softprob',n_jobs=-1).fit(X_train, y_train)
print('Accuracy of Xgb classifier on training set : {:.2f}'.format(ltv_xgb_model.score(X_train, y_train)))
print('Accuracy of Xbg classifier on test set: {:.2f}'.format(ltv_xgb_model.score(X_test[X_train.columns],y_test)))


Accuracy of Xgb classifier on training set : 0.99
Accuracy of Xbg classifier on test set: 0.82


In [ ]:
### Độ chính xác là 0.82 trên test. Các lớp dữ liệu chênh nhau rất nhiều. Cụm lớn nhất là cụm 0 chiếm 76,5%. xem thêm các chỉ số precision-recall 

In [47]:
y_pred = ltv_xgb_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.94      0.90        70
           1       0.58      0.39      0.47        18
           2       0.50      0.50      0.50         4

    accuracy                           0.82        92
   macro avg       0.65      0.61      0.62        92
weighted avg       0.80      0.82      0.80        92



#### Tỷ lệ dự đoán ở cụm 0 cao. Cụm 2 tỷ lệ recall cao hơn precision gần như không dự đoán đúng khách hàng ở cụm 1